<h1 style="font-family: Impact, Charcoal, sans-serif; padding: 12px; font-size: 30px; color: #8b4513; text-align: center; line-height: 1.25;">Sentiment Analysis<br><span style="color: #800000; font-size: 48px"><b>TripAdvisor Hotel Reviews</b></span><br><span style="color: #a0522d; font-size: 20px">Using Sklearn and Tensorflow</span></h1>

<div style="display: flex; justify-content: center;">
  <img src="assets/review.png" alt="Hotel Review" style="width:200px;">
</div>

<p>Data source: <a href="https://www.kaggle.com/datasets/thedevastator/tripadvisor-hotel-reviews">TripAdvisor Hotel Reviews</a></p>


<a id="6"></a>
<h1><b><span style="color: #8b4513; font-size: 28px">Building Model with Tensorflow</span></b></h1>

In [ ]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.regularizers import l1, l2

In [ ]:
from sklearn.preprocessing import LabelBinarizer
tokenizer = Tokenizer(num_words=50000, oov_token='<OOV>')

tokenizer.fit_on_texts(X_train)
# print(tokenizer.word_index)
total_word = len(tokenizer.word_index)
print('Total distinct words: {}'.format(total_word))

train_seq = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_seq)

test_seq = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_seq)

# One hot encoding the label
lb = LabelBinarizer()
train_labels = lb.fit_transform(y_train)
test_labels = lb.transform(y_test)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
tokenizer = Tokenizer(num_words=50000, oov_token='<OOV>')

tokenizer.fit_on_texts(X_train)
# print(tokenizer.word_index)
total_word = len(tokenizer.word_index)
print('Total distinct words: {}'.format(total_word))

train_seq = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_seq)

test_seq = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_seq)

# One hot encoding the label
lb = LabelBinarizer()
train_labels = lb.fit_transform(y_train)
test_labels = lb.transform(y_test)

In [ ]:
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))
pickle.dump(lb, open('label.pkl', 'wb'))

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Embedding(total_word, 8),
                                    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(64, kernel_regularizer=l2(0.001),
                                                          bias_regularizer=l2(0.001), activation='relu'),
                                    tf.keras.layers.Dense(3, activation='softmax')])

model.summary()

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_padded, train_labels, epochs=25, validation_data=(test_padded, test_labels))

In [ ]:
metrics = pd.DataFrame(model.history.history)
metrics[['accuracy', 'val_accuracy']].plot()
metrics[['loss', 'val_loss']].plot()

In [ ]:
pred2 = model.predict(test_padded)

In [ ]:
model.save('dl_model.h5')

<a id="7"></a>
<h1><b><span style="color: #8b4513; font-size: 28px">Predictions</span></b></h1>

In [ ]:
# XGBoost
def ml_predict(text):
    text = clean_text(text)
    tfid_matrix = tfid.transform([text])
    pred = best_model.predict(tfid_matrix)[0]
    
    return pred

# Deep Neural Network
def dl_predict(text):
    text = clean_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq)

    pred = model.predict(padded)
    # Get the label name back
    result = lb.inverse_transform(pred)[0]
    
    return result

In [ ]:
text = 'Such a comfy place to stay with the loved one'

print('Prediction using XGBoost: {}'.format(ml_predict(text)))
#print('Prediction using DNN: {}'.format(dl_predict(text)))

In [ ]:
text2 = 'Awful room services and slow wifi connection'

print('Prediction using XGBoost: {}'.format(ml_predict(text2)))
#print('Prediction using DNN: {}'.format(dl_predict(text2)))

In [ ]:
text3 = 'Hard to get here but the scenery is wonderful'

print('Prediction using Logistic Regression: {}'.format(ml_predict(text3)))
#print('Prediction using DNN: {}'.format(dl_predict(text3)))